# USING A ROUTE OPTIMIZATION NATIVE APP WITH AISQL


Using Cortex with  either the open route optimization service **API** or self contained **Native App**

#### 1.  Simple Directions - allows simple directions from one point to another.
You can load multiple start/end points to get directions for multiple places). You also need to specify the method such as driving-car.  This function is leveraging a GET request from the api.  You will note that the function is written in python - the service itself has handy python snippets to help curate the function.  
click [here](https://openrouteservice.org/dev/#/api-docs/v2/directions/{profile}/post) to navigate to the directions service playground

I have decided to use the **AI_COMPLETE** function with **Anthropic LLM** -  **Claude** to provide an array of values which contain one nice hotel and one nice restaurant in **New York City**.

In the following cell, indicate which functions you will be using.  If you are using functions from the native app, choose **OPENROUTESERVICE_NATIVE_APP**.  If you are using the external integration api calls, choose **VEHICLE_ROUTING_SIMULATOR**.

In [ ]:
#database = 'OPENROUTESERVICE_NATIVE_APP'
database = 'VEHICLE_ROUTING_SIMULATOR'

# populate the above with either the native app name or the database which 
# holds the api functions.  This is called VEHICLE_ROUTING_SIMULATOR
# If testing locally, the app name is called OPENROUTESERVICE_NATIVE_APP

## Add your secret if you are using the API

##ALTER SECRET VEHICLE_ROUTING_SIMULATOR.CORE.ROUTING_TOKEN SET SECRET_STRING = 'secret for open route service';



Firstly we will generate some data used to call the directions service

In [ ]:
CREATE or replace table SIMPLE_DIRECTIONS AS

SELECT AI_COMPLETE(


model =>'claude-4-sonnet', 

prompt =>'Return 1 hotel in Brooklyn and 1 restaurant in Manhattan IN NEW YORK CITY.',
response_format => {
            'type':'json',
            'schema':{'type' : 'object','properties' : {'locations':{'type':'array','items':{'type':'object','properties':
            {'place' : {'type' : 'string'},'longitude': {'type':'number'}, 'latitude': {'type':'number'}},'required':['place','longitude' ,'latitude']}}}}
            }
    
) PLACES;

select * from SIMPLE_directions

You will now use this data to call the directions function.  The function expects a string for the vehicle profile, and a variant for the start and end point

In [ ]:
select * exclude GEO, TO_GEOGRAPHY(GEO:features[0]:geometry) GEO,
GEO:features[0]:properties:segments INSTRUCTIONS, 
DIV0(GEO:features[0]:properties:summary:distance::FLOAT,1000) DISTANCE_KM,
DIV0(GEO:features[0]:properties:summary:duration::FLOAT,60) DURATION_MINS

from



(
select
{{database}}.CORE.DIRECTIONS('driving-car',
object_construct('coordinates',array_agg(array_construct(VALUE:longitude,
VALUE:latitude)))) GEO,
array_agg(VALUE:place) destinations

FROM SIMPLE_DIRECTIONS, lateral flatten (places:locations))

## Cortex Generated Map

I will now introduce you to the **pydeck** library.  This library allows you to visualise multiple map layers which can include polygons,linestrings and points.

In [ ]:
CREATE OR REPLACE TABLE CORTEX_MAP AS

select *, AI_COMPLETE('claude-4-sonnet',

CONCAT('Create a pandas dataframe using a snowpark dataframe called mapdf. the snowpark dataframe has one geography column called GEO.  use a lamda function with json.loads to extract the coordinates for a pydeck path layer to use for a pydeck path layer.  Create the pydeck path layer with a min line width of 5 and a colour of green and map.  add a tooltip that has DISTANCE_KM and DURATION_MINS both of which are columns in the pandas dataframe which was based on the mapdf snowpark dataframe.  Add place names from the DESTINATIONS column.  Create a scatter layer which has the start and end point.  Make sure pickable is switched off, the radius is a max size of 10 and a min size of 5, the scale is 1 and the color is black. render the result as a streamlit pydeck map. The map style is none.  RETURN THE PYTHON CODE ONLY'))::TEXT PYTHON from

{{use_directions_function}};

SELECT * FROM CORTEX_MAP

In [ ]:
import streamlit as st
from snowflake.snowpark.context import get_active_session
session = get_active_session()

st.markdown('### THIS IS A CORTEX GENERATED MAP')
code = session.table('CORTEX_MAP').select('PYTHON').collect()[0][0]

mapdf = use_directions_function.to_df()
code2 = code.replace("```python\n", "").replace("```", "")
with st.expander('Python Code for Map'):
    st.write(code)
with st.expander('The Pydeck Map'):
    exec(code2)

## Advanced directions - create routes which include multiple way points. 
Although the function above is nice and simple for directions that stops at only one place, vehicle route plans that contain multiple drop offs will need to effectively 'detour' between the starting point and destination.  This is where we utilise **way points**.

Each stop is loaded into one array.  You also need to specify the method such as driving-car.  This type of api uses a PUT request - you will see how this is articulated in the code below.

Lets now take you through calling the service which pushes in multiple way points.  Firstly, you will create a series of waypoints in an array.  Python is a great way to inject array variables into our sql function via a notebook for testing purposes.

In [ ]:
CREATE or replace table TEN_RANDOM AS 

select 

value:latitude LAT,
value:longitude LON,
value:restaurant_name::TEXT RESTAURANT_NAME,
array_construct(LON,LAT) COORDINATES


from (SELECT AI_COMPLETE(

model=>'claude-4-sonnet',

prompt=>'Return 10 restaurants in New York City.',
           
response_format =>   {
            'type':'json',
            'schema':{'type' : 'object','properties' : {'locations':{'type':'array','items':{'type':'object','properties':
            {'restaurant_name' : {'type' : 'string'},'longitude': {'type':'number'}, 'latitude': {'type':'number'}},'required':['restaurant_name','longitude' ,'latitude']}}}}
            }
    
) PLACES),lateral flatten (places:locations);

SELECT * FROM TEN_RANDOM

We will now run the function to include multiple way points

In [ ]:
select LON, LAT,{{database}}.CORE.DIRECTIONS

('driving-car',

object_construct('coordinates',coordinates)):features[0] all_data, 
all_data:geometry GEO, 
all_data:properties PROPERTIES

FROM 

(select AVG(LON) LON, AVG(LAT) LAT, array_agg(coordinates) coordinates from TEN_RANDOM)

## VISUALISE MULTI LAYERS
We will now visualise the the directions with way points again using pydeck.  This incorporates two layers - one for the line string and the second for each way point 

In [ ]:
select * EXCLUDE ALL_DATA from 


(select ARRAY_AGG(object_construct('coordinates',COORDINATES,'restaurant_name',RESTAURANT_NAME)) drop_offs 

from TEN_RANDOM) 

inner join {{way_points}}

In [ ]:
CREATE  or replace table CORTEX_MAP_2 AS

select *, AI_COMPLETE('claude-4-sonnet',

CONCAT('''

use an existing Snowpark dataframe called dropoffs_df,
This dataframe has a column called GEO.  This column has a datatype of Geography
convert the dataframe to pandas called dropoffs_pd 
Use a  lamda function with json.loads to transform the GEO column in the dropoffs_pd data frame to contain an array of longitude and latitude coordinates.  

In streamlit, create two tabs.  Tab 1 will contain a pydeck map.  generate the pydeck code needed to render a path layer map using the GEO column in the pandas dataframe.  the path layer should not be pickable. the size width of the path layer should be suitable to mark clear route directions on a map.  The map style is none. Apply the code to an st.pydeck_chart so import streamlit and use the GEO column, Apply a second layer which is a scatter layer.  This has all the points using the dropoffs column,  the drop offs column is an array containing multiple objects.  one object is called coordinates which is an array containing a longitude and latitude.  another object is the restaurant_name.Make sure pickable is switched on, the radius is a max size of 10 and a min size of 5, the scale is 1 and the color is black.  Add an appopiate tooltip for each point.  

Tab 2 will contain markdown which renders the first row the following direction data for all segments. The segments are in a column called PROPERTIES, the restaurant data is in a field called DROP_OFFS and the path layer data is in a field called GEO.  you will need to use json.loads to pars the json in GEO, PROPERTIES and DROPOFFS.  this data frame will be available before the code is ran''', properties::text, ''' return python only'''))::TEXT PYTHON from

{{all_data_2}};


In [ ]:
import streamlit as st
from snowflake.snowpark.context import get_active_session
session = get_active_session()

dropoffs_df = all_data_2.to_df()

code = session.table('cortex_map_2').select('PYTHON').collect()[0][0]

code2 = code.replace("```python\n", "").replace("```", "")

with st.container(height=800):
    with st.expander('Python Code for Map'):
        st.write(code)
    with st.expander('The Pydeck Map'):
        exec(code2)

## ROUTE OPTIMIZATION FUNCTION

  

If you know the jobs and know what vehicles are availalble, whats the most optimal way of getting the goods/services there??

In [ ]:
CREATE OR REPLACE TABLE supplier AS

SELECT AI_COMPLETE(

        model=>'claude-4-sonnet', 
        prompt=>'give me a location in new york city that sells food to restaurants.            return the name of the supplier and the latitude and longitude',
        response_format =>{
            'type':'json',
            'schema':{'type' : 'object','properties' : {'locations':{'type':'array','items':{'type':'object','properties':
            {'place' : {'type' : 'string'},'longitude': {'type':'number'}, 'latitude': {'type':'number'}},'required':['place','longitude' ,'latitude']}}}}
            }
    
):locations[0] SUPPLIER,

supplier:latitude::float lat, supplier:longitude::float lon, supplier:place::text restaurant;

select * from SUPPLIER

## CREATE A JOB AND VEHICLE TEMPLATE TO HELP CORTEX SIMULATE JOBS AND AVALABLE VEHICLES
Below is a sample format for the jobs and vehicles and is an illustration of what the optimization service expects.

In [ ]:
vehicles =  [
  {
    "capacity": [
      2
    ],
    "setup":120,
    "service":240,
    "id": 1,
    "location": [
      -4.613748000000000e-01,
      5.152569760000000e+01
    ],
    "skills": [
      1
    ],
    "time_window": [
      'epoch format',
      'epoch format'
    ]
  },
  {
    "capacity": [
      2
    ],
    "id": 6,
    "location": [
      -3.355460000000000e-01,
      5.150566550000000e+01
    ],
    "skills": [
      1
    ],
    "time_window": [
      36000,
      54000
    ]
  }

]

jobs = [
  {
    "capacity": [
      2
    ],
    "end": [
      -4.467074000000000e-01,
      5.146048020000000e+01
    ],
    "id": 1,
    "profile": "driving-hgv",
    "skills": [
      1
    ],
    "start": [
      -4.467074000000000e-01,
      5.146048020000000e+01
    ],
    "time_windows": [
      'epoch format',
      'epoch format'
    ]
  },
]
col1, col2 = st.columns(2)

with col1:
    st.markdown('### Format for Jobs')
    st.json(jobs)
with col2:
    st.markdown('### Format for Vehicles')
    st.json(vehicles)

We will now create a synthetic vehicle as well as 10 delivery jobs that this vehicle needs to cover in New York.  We are specifying Structured json to generate the jobs and vehicle

In [ ]:
CREATE or replace table OPTIMIZATION_1_VEHICLE AS

SELECT *,AI_COMPLETE(

model=>'claude-4-sonnet', 

prompt=> CONCAT('Return 10 delivery jobs with 1 available vehicle in New York City. on the 4th june 2025.  Make delivery routes based this data. ',drop_offs::text, 'all time_windows are in epoch format.  all vehicles will start from this supplier',supplier::text,'  this simulation will be used for a route optimization service.  the vehicle profile will be driving-car.  location is an array containing two values, longitude and latitude.' ),
            
    
model_parameters=>    {
        'temperature': 0,
        'max_tokens': 5000},

response_format=>{
            'type':'json',
            'schema':{'type' : 'object','properties' :
{'job_vehicles':{'type':'array','items':

{'jobs':{'type':'array','items':{'type':'object','properties':
            {'capacity' : {'type' : 'array'},
            'setup': {'type':'number'},
            --'service': {'type':'number'},
            'id':{'type':'number'}, 
            'location':{'type':'array'},
            'skills':{'type':'array'},
            'time_window':{'type':'array'}
            },
            'required':['capacity','setup','skills','time_window''id','location']}},
            
'vehicles':{'type':'array','items':{'type':'object','properties':
            {'capacity' : {'type' : 'array'},
            'start': {'type':'array'},
            'end': {'type':'array'},
            'service': {'type':'number'},
            'id':{'type':'number'}, 
            'location':{'type':'array'}, 
            'skills':{'type':'array'},
            'profile':{'type':'string'},
            'time_window':{'type':'array'}
            },'required':['capacity','start','end','service','id','location','profile','time_window']
            }}
            
            }
   
    }}}}
) JOBS  FROM

(select * from (select array_agg(drop_offs) drop_offs from {{all_data_2}}) inner join supplier);

SELECT * FROM OPTIMIZATION_1_VEHICLE

## PUT THE JOBS AND VEHICLES THROUGH THE ROUTE OPTIMIZATION SERVICE
Let's now put the jobs through the route optimization service

In [ ]:
select JOBS:job_vehicles[0]:vehicles,JOBS:job_vehicles[0]:jobs from OPTIMIZATION_1_VEHICLE

In [ ]:
select *, {{database}}.CORE.OPTIMIZATION(
    JOBS:job_vehicles[0]:jobs, 
   JOBS:job_vehicles[0]:vehicles[0]) 
 OPTIMIZATION from OPTIMIZATION_1_VEHICLE

Let's now have a look at the json object. You will see that the array object is broken down into routes.  Each route will be assigned a vehicle and that vehicle will be allocated delivery locations and times.  There will also be infomation about the duration and may include details such as waiting times.

If you used the native app, it will include a geometry object which gives you an array of every point for each route.  You will not see this if you use the external api calls.  However, you can still achieve similar results by using the directions function for each route.

In [ ]:
with st.container(height=500):
    st.code(test_route_optimization.to_df().to_pandas().OPTIMIZATION.iloc[0])

View the step line and the directions line with the points on a map.

**Note** If you are using the functions from an api, the code below will call the directions api to get the line for route path.  The native app provides this as part of the optimization api.

You will see logic in the code to call the directions service if no geometry data is provided by the optimization service.

In [ ]:
import streamlit as st
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import *
from snowflake.snowpark.types import *
import pandas as pd
import pydeck as pdk
import json

from snowflake.snowpark.functions import *
from snowflake.snowpark.types import *

routes = test_route_optimization.to_df()
routes = routes.with_column('steps',col('OPTIMIZATION')['routes'][0]['steps'])

routes = routes.with_column('GEOMETRY',col('OPTIMIZATION')['routes'][0]['geometry'])

api = 'no'
if routes.select('GEOMETRY').collect()[0][0] is None:
    api = 'yes'
    routes = routes.drop('GEOMETRY')
    steps = routes\
    .join_table_function('flatten',col('OPTIMIZATION')['routes']).\
    with_column_renamed('VALUE','LEV1').\
    join_table_function('flatten',col('LEV1')['steps']).select(object_construct(lit('coordinates'),
                                                                                array_agg(col('VALUE')['location'])).alias('STEPS'))


    steps = steps.select(call_function(f'''{database}.CORE.DIRECTIONS''',lit('driving-car'),col('STEPS'))['features'][0]['geometry']['coordinates'].alias('GEOMETRY'))
    routes = routes.join(steps)
 

routes = routes.join_table_function('flatten',col('steps')).select('VALUE','SUPPLIER','GEOMETRY')
routes = routes.select('GEOMETRY',
              to_char(to_timestamp(col('VALUE')['arrival'])).alias('ARRIVAL'),
              col('VALUE')['duration'].alias('duration'),
              col('VALUE')['distance'].alias('distance'),
              col('VALUE')['setup'].alias('setup'),
              col('VALUE')['id'].alias('Job ID'),
              parse_json('VALUE')['location'][1].astype(FloatType()).alias('LAT'),
              parse_json('VALUE')['location'][0].astype(FloatType()).alias('LON'),
              col('VALUE')['service'].alias('Service'))
            
line = routes.select(any_value('GEOMETRY').alias('GEOMETRY'),call_function('ST_COLLECT',call_function('st_makepoint',col('LON'),col('LAT'))).alias('LINE')) 
line = line.with_column('CENTROID',call_function('ST_CENTROID',(call_function('ST_ENVELOPE',col('LINE')))))
routes = routes.drop('GEOMETRY')


tooltip = {
   "html": """<b>Job ID:</b> {Job ID}
   <br><b>Arrival:</b> {ARRIVAL} 
   <br> <b>Duration:</b> {DURATION}
   <br> <b>Setup:</b> {SETUP}
   <br> <b>Service:</b> {SERVICE}
   <br> <b>Distance:</b> {DISTANCE}""",
   "style": {
       "width":"50%",
        "backgroundColor": "steelblue",
        "color": "white",
       "text-wrap": "balance"
   }
}



session = get_active_session()

center = line.select(call_function('ST_X',col('CENTROID')).alias('X'),
                     call_function('ST_Y',col('CENTROID')).alias('Y')).limit(1).to_pandas()

LAT = center.Y.iloc[0]
LON = center.X.iloc[0]


directions = line.select(object_construct(lit('coordinates'),col('GEOMETRY')).alias('GEOMETRY'))
routespd = routes.to_pandas()

data = directions.select('GEOMETRY').to_pandas()
    
data["coordinates"] = data["GEOMETRY"].apply(lambda row: json.loads(row)["coordinates"])


st.write(data)
data2 = line.to_pandas()
data2["coordinates"] = data2["LINE"].apply(lambda row: json.loads(row)["coordinates"])
data3 = session.table('supplier').to_pandas()
st.write(data3)


sroutes = pdk.Layer(
        'ScatterplotLayer',
        data=routespd,
        get_position=['LON','LAT'],
        get_radius=10,
        radius_min_pixels=5,
        radius_max_pixels=10,
        radius_scale=1,
        get_color=[0, 0, 0],
        pickable=True
    )

 # Create scatter layer for restaurants
slayer = pdk.Layer(
        'ScatterplotLayer',
        data=data3,
        get_position=['LON', 'LAT'],
        get_radius=30,
        radius_min_pixels=5,
        radius_max_pixels=30,
        radius_scale=1,
        get_color=[41, 181, 232],
        pickable=False
    )

vehicle_1_path = pdk.Layer(
type="PathLayer",
data=data,
pickable=False,
get_color=[253, 180, 107],
width_scale=20,
width_min_pixels=4,
width_max_pixels=7,
get_path="coordinates",
get_width=5)

vehicle_1_straight_line = pdk.Layer(
type="PathLayer",
data=data2,
pickable=False,
get_color=[255,0, 107],
width_scale=20,
width_min_pixels=4,
width_max_pixels=7,
get_path="coordinates",
get_width=5)

view_state = pdk.ViewState(latitude=LAT, longitude=LON, zoom=10)
st.markdown('Waypoints joined together as a straight line')
st.pydeck_chart(pdk.Deck(tooltip=tooltip,layers=[vehicle_1_path,vehicle_1_straight_line,slayer,sroutes],map_style=None,initial_view_state=view_state,height=900))

## Multiple Vehicles 
we will modify the data to have more jobs so now more vehicles are needed.  Firstly, let's create some quick sample data for 50 customers in New York City

In [ ]:
-- Create or replace the 'customers' table
CREATE OR REPLACE TABLE sample_customers AS
SELECT
    -- Generate a unique customer ID using ROW_NUMBER()
    ROW_NUMBER() OVER (ORDER BY customer_data.index) AS ID,
    -- Extract the name from the flattened customer object
    customer_data.value:name::TEXT AS name,
    -- Extract the address from the flattened customer object
    customer_data.value:address::TEXT AS address,
    customer_data.value:email::TEXT AS email,
    -- Extract the latitude from the flattened customer object, casting to float
    customer_data.value:latitude::FLOAT AS lat,
    -- Extract the longitude from the flattened customer object, casting to float
    customer_data.value:longitude::FLOAT AS lon
FROM
    (
        
        SELECT
            AI_COMPLETE(
            model=> 'claude-4-sonnet',
            prompt=> 'give me 50 random residential locations in New York City for a               customer. return the name of the person, their address, a random email and              and the latitude and longitude',
            model_parameters=>        
                
                {
                    'temperature': 1,
                    'max_tokens': 5000},
            response_format=> {
                        'type': 'json',
                        'schema': {
                            'type': 'object',
                            'properties': {
                                'locations': {
                                    'type': 'array',
                                    'items': {
                                        'type': 'object',
                                        'properties': {
                                            'name': {'type': 'string'},
                                            'address': {'type': 'string'},
                                            'email': {'type': 'string'},
                                            'longitude': {'type': 'number'},
                                            'latitude': {'type': 'number'}
                                        },
                                        'required': ['name', 'address','email','longitude', 'latitude']
                                    }
                                }
                            }
                        }
                    }
                
            ):locations AS locations_array -- Alias the extracted array
    ) AS cortex_output, -- Alias the subquery result
    -- Use LATERAL FLATTEN to unnest the 'locations_array' into separate rows
    LATERAL FLATTEN(input => cortex_output.locations_array) AS customer_data;

-- Select all data from the newly created/replaced 'customers' table
SELECT * FROM sample_customers;


You will now create an object for all the drop offs and an object for the starting location which is the Restaurant

In [ ]:
SELECT * FROM (SELECT 
array_agg(OBJECT_CONSTRUCT('Full Name',NAME,'ID',ID,'LAT',LAT,'LON',LON,'ADDRESS',ADDRESS)) DROP_OFFS 
FROM sample_customers) A 

INNER JOIN


(select drop_offs[6] RESTAURANT from {{all_data_2}}) B

Now lets generate the Jobs and vehicles

In [ ]:
number_of_vehicles = 3  #the optimization api service has a maximum of 3 vehicles. 
## Try 5 if running the native app.

#number_of_vehicles = 5

In [ ]:
CREATE OR REPLACE TABLE MULTI_VEHICLE_OPTIMIZATION AS

SELECT *,AI_COMPLETE(

model => 'claude-4-sonnet',

prompt => CONCAT('create 1 object containing take away food deliveries to customers where all take aways need to be delivered on the 4th june 2025 at random and unique time slots between 1800 and 1900 using this data''', drop_offs::text, 'in New York City based on the following template where the time windows are converted to epoch format,  Vary the setup time to be between 120 and 560 seconds and use the ID column for the Job ID.  the start and end points are based on the following json',restaurant::text, 'which has service time of either 240 or 360 seconds.  the skills value for the vehicles are either 10 or 20.  the skills value for the jobs are either 10 or 20.  Also create 1 object containing', {{number_of_vehicles}} ,' vehicles with the profile of driving-car that will start work on the 4th June 2025 at 1730 and will need to return back to the restaurant at 2100 on the same day. populate the start and end arrays to reflect this.  the start array is an array of longitude and latitude coordinates.  the end array is an array of longitude and latitude coordinates.  the time windows are in epoch format.  location is an array containing two numeric values, longitude and latitude.' ),
model_parameters=>
    {
        'temperature': 0,
        'max_tokens': 8000},
response_format=>{
            'type':'json',
            'schema':{'type' : 'object','properties' :
{'job_vehicles':{'type':'array','items':

{'jobs':{'type':'array','items':{'type':'object','properties':
            {'capacity' : {'type' : 'array'},
            'setup': {'type':'number'},
            --'service': {'type':'number'},
            'id':{'type':'number'}, 
            'location':{'type':'array'},
            'skills':{'type':'array'},
            'time_window':{'type':'array'}
            },
            'required':['capacity','setup','skills','time_window''id','location']}},
            
'vehicles':{'type':'array','items':{'type':'object','properties':
            {'capacity' : {'type' : 'array'},
            'start': {'type':'array'},
            'end': {'type':'array'},
            'service': {'type':'number'},
            'id':{'type':'number'}, 
            'location':{'type':'array'}, 
            'skills':{'type':'array'},
            'profile':{'type':'string'},
            'time_window':{'type':'array'}
            },'required':['capacity','skills','start','end','service','id','location','profile','time_window']
            }}
            
            }
   
    }}}}
) JOBS  FROM

(select *  from {{take_away}}) ;

SELECT *, jobs:job_vehicles[0]:jobs, jobs:job_vehicles[0]:vehicles FROM MULTI_VEHICLE_OPTIMIZATION 

Lets now expand the jobs and vehicles so we can see what has been created.

In [ ]:
import streamlit as st
from snowflake.snowpark.context import get_active_session
session = get_active_session()

from snowflake.snowpark.functions import *

vehicles = session.table('MULTI_VEHICLE_OPTIMIZATION').join_table_function('flatten',col('JOBS')['job_vehicles'][0]['vehicles']).select('VALUE')  #select value from {{take_away_deliveries}}, lateral flatten (JOBS:vehicles)
jobs = session.table('MULTI_VEHICLE_OPTIMIZATION').join_table_function('flatten',col('JOBS')['job_vehicles'][0]['jobs']).select('VALUE')

col1,col2 = st.columns(2)

with col1:
    jobs
with col2:
    vehicles


You will now run the optimization service based on these jobs and available vehicles.  Again, having the geometry field present will depend on whether you chose to use the native app or the external integration api functions.

In [ ]:
select *, {{database}}.CORE.OPTIMIZATION(
    JOBS:job_vehicles[0]:jobs, 
   JOBS:job_vehicles[0]:vehicles) 
 OPTIMIZATION from MULTI_VEHICLE_OPTIMIZATION

this sql runs the directions service if there is no geometry provided.  If the native app is used, this part will be skipped.

In [ ]:
WITH FlattenedData AS (
    
    select a.*,b.profile from 
    (SELECT
        restaurant:coordinates[0]::float AS lon,
        restaurant:coordinates[1]::float AS lat,
        value:vehicle AS VEHICLE_ID,
        value:cost AS COST,
        value:duration AS DURATION,
        value:distance AS DISTANCE,
        value:geometry AS GEOMETRY, -- Original GEOMETRY column
        value:steps AS STEPS         -- Original STEPS column (assumed to be an array of objects)
    FROM
        {{optimization_takeaway}},
    LATERAL FLATTEN (optimization:routes)) a inner join

    (select value:id ID, 
            value:profile::TEXT PROFILE 
            from multi_vehicle_optimization,
    lateral flatten(JOBS:job_vehicles[0]:vehicles)) b 
    
    on a.VEHICLE_ID = b.ID
)
SELECT
    lon,
    lat,
    VEHICLE_ID,
    COST,
    STEPS,
    DURATION,
    DISTANCE,
    PROFILE,
    CASE
        WHEN GEOMETRY IS NULL THEN
            -- Use REDUCE to aggregate all 'location' objects from the STEPS array into a new array.
            {{database}}.core.directions(PROFILE,OBJECT_CONSTRUCT('coordinates',REDUCE(
                STEPS,
                ARRAY_CONSTRUCT(), -- Initial state: an empty array
                (acc, curr) -> ARRAY_APPEND(acc, curr:location)
            ))):features[0]:geometry:coordinates
        ELSE GEOMETRY -- Use the original GEOMETRY when it's not NULL
    END AS GEOMETRY -- This column will now contain an array of locations if original GEOMETRY was NULL
FROM
    FlattenedData;


In [ ]:
CREATE  or replace table PYTHON_MULTI_VEHICLE_CODE AS

select replace(replace(AI_COMPLETE('claude-4-sonnet', '''


use the supplied pandas dataframe called routesdf.

create a record called location which is the first row from the routesdf dataframe.

create a variable called LAT which is a column called LAT in location. this is a float data type. 
create a variable called LON which is a column called LON in location.  this is a float data type. 

create a streamlit mulit selector named select_route to filter each vehicle.  valid values are in the VEHICLE_ID column. Each value is numeric.  Add a default value which will be the first available column.

create a dataframe called filtered_route_df by filtering the routesdf dataframe based on the select_route selector.

create a new column called color and add a unique colour in RGB format.  This needs to be in an array.

transform the GEOMETRY field by converting the contents into an array using json.loads.


Create a path layer.  Each path represents a vehicle is coming from rows within the filtered_route_df dataframe.   the column name will be called GEOMETRY the data format for each line is like this [[lon, lat],[lon,lat]].  set pickable to false on this layer.  Color each path based on the color field.


Select the first value for both LON and LAT.  Each row of the data represents a vehicle.  

create a second layer which is a scatter plot layer which shows the resturant location.  this is determined by the the variables LAT and LON.  each variable DOES NOT need to be parsed. set this layer as not pickable.

create a dataframe called stepsdf which needs to be flattened.  get the data from the STEPS column in filtered_route_df. parse the column and flatten to reveal multiple records. create a column for each object.  keep the VEHICLE_ID in the dataframe.
the position is in a location column that looks like this [lon,lat]. the same data frame will have a column called id.

join this data frame with a data frame called people and join by the id column.  put the people columns first and the steps data second.


create a third layer which is a scatter plot layer which uses the stepdf dataframe.  get this data from the stepsdf.  Colour each point the same colour as their delivery vehicle. make this layer pickable


create a tooltip to use with the scatter plot layer which will create a heading and value nicely formatted containing all the information from the stepdf dataframe.  

give the tool tip a heading called job details. Ensure the parameters only have 3 braces.  make the background color blue and slightly transparent.

create a view state which uses the variable LAT and LON and the map style is None.

Under the map, create a legend to show the color of each vehicle.  state the PROFILE for each vehicle.  this is named in in the routesdf column called PROFILE.

return only python''' )::TEXT,'```',''),'python','') PYTHON;

SELECT * FROM PYTHON_MULTI_VEHICLE_CODE

In [ ]:
from snowflake.snowpark.functions import *
routesdf = multi_vehicle.to_pandas()
people = session.table('sample_customers').select(col('ID').alias('"id"'),col('NAME'),col('EMAIL')).to_pandas()

python = session.table('PYTHON_MULTI_VEHICLE_CODE').collect()[0][0]

with st.expander('Python Code'):
    st.code(python)
exec(python)

## Isochrone Function.  
This is to create a polygon which creates a catchment boundary based on how long it would take to get there.  Orders may only be accepted if you can fullfill them based on how quickly drivers can get anywhere within a given timeframe.  Isochrones could be utilised for other usecases outside of route optimization such as market analysis in a given catchment.

The function is then tested by focussing on a specific point.  It will generate a polygon which represents everywhere a driver can get to within a 20 minute timeframe.

In [ ]:
CREATE or replace table GEOCODE_NEW_YORK_OFFICE AS 
SELECT parse_json(replace(replace(AI_COMPLETE('claude-4-sonnet', 'give me the lat and lon for the snowflake office - 114 W 41st St, Suite 1701, New York, NY 10036.  return the result as a json string'),'```',''),'json','')) NEW_YORK_OFFICE;

SELECT * FROM GEOCODE_NEW_YORK_OFFICE

In [ ]:
CREATE or replace table GEOCODE_NEW_YORK_OFFICE AS 
SELECT parse_json(replace(replace(AI_COMPLETE('claude-4-sonnet', 'give me the lat and lon for the snowflake office - 114 W 41st St, Suite 1701, New York, NY 10036.  return the result as a json string'),'```',''),'json','')) NEW_YORK_OFFICE;


CREATE OR REPLACE TABLE ISOCHRONE_DRIVING AS 

select 

NEW_YORK_OFFICE:longitude::float LON,
NEW_YORK_OFFICE:latitude::float LAT,
NEW_YORK_OFFICE:address::text ADDRESS,
{{database}}.CORE.ISOCHRONES('cycling-road', LON,LAT, 20) DATA_CYCLE, 

{{database}}.CORE.ISOCHRONES('driving-car', LON,LAT, 20) DATA_CAR, 

TO_GEOGRAPHY(DATA_CYCLE:features[0]:geometry) GEO_CYCLE, 
TO_GEOGRAPHY(DATA_CAR:features[0]:geometry) GEO_CAR, 
ST_MAKEPOINT(LON,LAT) CENTROID

from GEOCODE_NEW_YORK_OFFICE
;

SELECT * FROM ISOCHRONE_DRIVING

This is an example of what an isochrone looks like in pydeck.

In [ ]:
import streamlit as st
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import *
from snowflake.snowpark.types import *
import pandas as pd
import pydeck as pdk
import json
session = get_active_session()


data = isochrones_try.to_df().select('GEO_CYCLE','GEO_CAR','LON','LAT','ADDRESS').to_pandas()
LAT = data.LAT.iloc[0]
LON = data.LON.iloc[0]
data["cyclecoords"] = data["GEO_CYCLE"].apply(lambda row: json.loads(row)["coordinates"])
data["carcoords"] = data["GEO_CAR"].apply(lambda row: json.loads(row)["coordinates"])

isochrone = pdk.Layer(
"PolygonLayer",
data,
opacity=0.2,
get_polygon="cyclecoords",
filled=True,
get_line_color=[17,86,127],
get_fill_color=[250, 203, 156],
get_line_width=10,
line_width_min_pixels=4,
auto_highlight=True,
pickable=False)

isochrone2 = pdk.Layer(
"PolygonLayer",
data,
opacity=0.5,
get_polygon="carcoords",
filled=True,
get_line_color=[41,181,232],
get_fill_color=[200,230,242],
get_line_width=10,
line_width_min_pixels=6,
auto_highlight=True,
pickable=False)

supplier = pdk.Layer(
    'ScatterplotLayer',
    data=data,
    pickable=True,
    filled=True,
    stroked=True,
    opaciity=0.1,
    get_position=['LON', 'LAT'],
    get_color=[255, 0, 0],
    line_width_min_pixels=2,
    get_line_color=[0, 0, 0],
    get_radius=300,
    
)
view_state = pdk.ViewState(latitude=LAT, longitude=LON, zoom=10)
st.pydeck_chart(pdk.Deck(layers=[isochrone2,isochrone,supplier],tooltip={"text":'Address: {ADDRESS}'},map_style=None,initial_view_state=view_state,height=900))

## CARTO OVERTURE MAPS PLACES DATA FOR NEW YORK

Create a base table with search optimization to filter quickly on categories and geographies

Here we are using the places dataset which is provided by carto overture maps.  We are copying the data to add search optimization on the categories of place as well as the geography.  We will be leveraging the 'Search' function in the app to search through columns that may be relevant to the selected industry.

In [ ]:
CREATE OR REPLACE TABLE NEW_YORK_DATA AS SELECT * FROM OVERTURE_MAPS__PLACES.CARTO.PLACE

WHERE ST_GEOHASH(GEOMETRY,2) = 'dr';

SELECT * FROM NEW_YORK_DATA LIMIT 10;

In [ ]:
CREATE TABLE IF NOT EXISTS VEHICLE_ROUTING_SIMULATOR.DATA.PLACES as 

select 

any_value(GEOMETRY) GEOMETRY,

PHONES:list[0]['element']::text PHONES,
CATEGORIES:primary::text CATEGORY,
NAMES:primary::text NAME,
ADDRESSES:list[0]['element'] ADDRESS,

array_agg(value:element) as ALTERNATE 

from

(SELECT PHONES,CATEGORIES,NAMES,ADDRESSES,GEOMETRY,

categories:alternate:list AS LIST

from NEW_YORK_DATA), 

--where CATEGORIES:primary is not null),
LATERAL FLATTEN(LIST) GROUP BY ALL;

ALTER TABLE DATA.places ADD SEARCH OPTIMIZATION ON EQUALITY(ALTERNATE);
ALTER TABLE DATA.places ADD SEARCH OPTIMIZATION ON GEO(GEOMETRY);

SELECT * FROM DATA.PLACES limit 3

In [ ]:
sql = '''select * from ( select ST_X(GEOMETRY) LON, ST_Y(GEOMETRY) LAT from VEHICLE_ROUTING_SIMULATOR.DATA.PLACES where category = 'hotel') sample (5000 rows)'''

st.map(session.sql(sql))

In [ ]:
CREATE OR REPLACE TABLE HOTELS_IN_ISOCHRONE AS 
select A.* EXCLUDE (GEOMETRY),B.* EXCLUDE(GEO_CAR),ST_X(GEOMETRY) LON,ST_Y(GEOMETRY) LAT, B.GEO_CAR GEO 


from VEHICLE_ROUTING_SIMULATOR.DATA.PLACES A INNER JOIN


(select GEO_CAR from ISOCHRONE_DRIVING) B ON

ST_WITHIN(TO_GEOGRAPHY(A.GEOMETRY),TO_GEOGRAPHY(B.GEO_CAR))

where CATEGORY = 'hotel';

select * from HOTELS_IN_ISOCHRONE

Here, we are using the Scatter Plot layer to only view places by the isochrone 

In [ ]:
prompt = '''

        build a multi layer pydeck map python code.
        
        layer 1 - this is a polygon layer.  there is a table called ISOCHRONE_DRIVING. 
        Retrieve the table using a snowpark data frame and formulate it appropiately to create
        a polygon layer.  in this table use the column called GEO_CAR.  this column has a geography datatype. 
        convert this into a pandas dataframe

        use a lambda function to extract the coordinates from the GEO_CAR column which will now be a string in pandas.
        
        This layer is NOT pickable.

        layer 2 - this is a polygon layer.  use the same table as layer 1 called ISOCHRONE_DRIVING. 
        Retrieve the table using a snowpark data frame and formulate it appropiately to create
        a polygon layer.  This time, use the column called GEO_CYCLE.  this column has a geography datatype. 
        convert this into a pandas dataframe

        use a lambda function to extract the coordinates from the GEO_CYCLE column which will now be a string in pandas.
        
        This layer is NOT pickable.
        
        layer 3 - this is a scatterplot layer.  
        there is a table called HOTELS_IN_ISOCHRONE.  in here there is a column called LAT and a column called LON.
        size the points so they pinpoint hotels on a map easily within the city of New York.
        return the map as a streamlit map.  the session is the active session as it will be run in Snowflake in Streamlit.
        The map style is none
        This layer IS pickable
        Return only the python
        
        For layer 3 create a tooltip.  the tool tip will have the following: 
        The hotel name using the column NAME 
        The address which is in a json variant in a column called ADDRESS.  
        Use snowpark to extract out all the columns called country, freeform, locality, postcode and region
        change the datatype to all the address columns to StringType
        in the
        Address object into text columns and add all values to the tooltip.
        The phone number which is in a column called PHONES
        use icons in the tooltip where appropiate.
        Ensure all text within the tooltip is visible.
        when the right columns are extracted in snowpark convert to a pandas data frame and use this for the layer
        
        Format the look and feel to make it look like the map was made for the
        Snowflake World Tour Event in New York 2025.  make the polygon layer visible
        render the height of the st.pydeck as 900
        add a title relating to snowflake
        add a description regarding viewing hotels near by.
        MAKE ALL ALIASES IN UPPER CASE
        
        do not add cache data in the streamlit code'''

df = session.create_dataframe([{'prompt':prompt}])

df = df.with_column('python',call_function('AI_COMPLETE','claude-4-sonnet',col('prompt')).astype(StringType()))
df.write.mode('overwrite').\
save_as_table("ISOCHRONE_PYTHON_CODE")
df = session.table('ISOCHRONE_PYTHON_CODE')

In [ ]:
df1 = df.select(replace(replace('PYTHON',lit('```'),lit('')),lit('python'),lit('')))
with st.expander('code'):
    st.code(df1.collect()[0][0])

with st.container(height=1000):
    exec(df1.collect()[0][0])

## Create Base Data Table for App
Now we will focus on putting components together for a streamlit application.  We are aiming to create a dynamic routing analyser.  For this, we will need to generate jobs, vehicles and depots based on any location in the world.  Carto overture maps will help as it contains points of interests anywhere in the world. Cortex complete is also used for a simple 'geocoding' mechanism where a user will type in a key word and cortex will generate the coordinates that matches what central location the user wants to base the routing against

#### 1. Job Template
This template will be used to simulate jobs.  You can change the base values in this notebook - which will effect what sample jobs will be generated.  This template generates a sample of 29 jobs.  The location of these jobs and industry will be dynamic and depends on user interaction with the app.

In [ ]:
CREATE OR REPLACE TABLE VEHICLE_ROUTING_SIMULATOR.DATA.JOB_TEMPLATE (

ID INT AUTOINCREMENT PRIMARY KEY,
slot_start INT NOT NULL,
slot_end INT,
skills INT,
product STRING,
status STRING DEFAULT 'active'

);


INSERT INTO VEHICLE_ROUTING_SIMULATOR.DATA.JOB_TEMPLATE (slot_start, slot_end, skills, product, status) VALUES
(9, 10, 1, 'pa', 'active'),
(11, 15, 2, 'pb', 'active'),
(16, 18, 2, 'pb', 'active'),
(11, 13, 3, 'pc', 'active'),
(7, 16, 3, 'pc', 'active'),
(10, 15, 2, 'pa', 'active'),
(10, 15, 2, 'pa', 'active'),
(7, 16, 1, 'pa', 'active'),
(9, 18, 2, 'pb', 'active'),
(13, 18, 2, 'pb', 'active'),
(13, 18, 2, 'pb', 'active'),
(13, 18, 1, 'pa', 'active'),
(13, 18, 1, 'pa', 'active'),
(13, 18, 1, 'pa', 'active'),
(13, 18, 3, 'pc', 'active'),
(11, 15, 2, 'pb', 'active'),
(16, 18, 2, 'pb', 'active'),
(11, 13, 1, 'pa', 'active'),
(7, 16, 1, 'pa', 'active'),
(10, 15, 2, 'pb', 'active'),
(10, 15, 2, 'pb', 'active'),
(7, 16, 1, 'pa', 'active'),
(9, 18, 2, 'pb', 'active'),
(13, 18, 2, 'pb', 'active'),
(13, 18, 2, 'pb', 'active'),
(13, 18, 1, 'pa', 'active'),
(13, 18, 1, 'pa', 'active'),
(13, 18, 1, 'pa', 'active'),
(13, 18, 3, 'pc', 'active');

SELECT * FROM VEHICLE_ROUTING_SIMULATOR.DATA.JOB_TEMPLATE

#### Below are all the categories you could use to simulate distribution points
The app will search through names, categories and alternative categories in order to retrieve the relevant data

In [ ]:
SELECT DISTINCT CATEGORY FROM (
select DISTINCT VALUE::TEXT CATEGORY FROM VEHICLE_ROUTING_SIMULATOR.DATA.PLACES, LATERAL FLATTEN (ALTERNATE)

UNION 

SELECT DISTINCT CATEGORY FROM VEHICLE_ROUTING_SIMULATOR.DATA.PLACES) WHERE SEARCH((CATEGORY),'food')

### 4. Creating an Industry Lookup

Depending on the selected industry, the app will 'lookup' specifics in order to retrieve/ filter the right information.  It is also used to generate relevant product categories. In this notebook, more industries can be added.

In [ ]:
CREATE OR REPLACE TABLE VEHICLE_ROUTING_SIMULATOR.DATA.LOOKUP (
    INDUSTRY STRING,
    PA STRING,
    PB STRING,
    PC STRING,
    IND ARRAY,
    IND2 ARRAY,
    CTYPE ARRAY,
    STYPE ARRAY
);

INSERT INTO VEHICLE_ROUTING_SIMULATOR.DATA.LOOKUP (INDUSTRY, PA, PB, PC, IND,IND2, CTYPE, STYPE) 
SELECT
    'healthcare', 
    'flammable', 
    'sharps', 
    'temperature-controlled', 
    ARRAY_CONSTRUCT('hospital health pharmaceutical drug healthcare pharmacy surgical'), 
    ARRAY_CONSTRUCT('supplies warehouse depot distribution wholesaler distributors'), 
    ARRAY_CONSTRUCT('hospital', 'family_practice', 'dentist','pharmacy'), 
    ARRAY_CONSTRUCT('Can handle potentially explosive goods', 'Can handle instruments that could be used as weapons', 'Has a fridge')
UNION ALL
SELECT
    'Food', 
    'Fresh Food Order', 
    'Frozen Food Order', 
    'Non Perishable Food Order', 
    ARRAY_CONSTRUCT('food vegatables meat vegatable'),
    ARRAY_CONSTRUCT('wholesaler warehouse factory processing distribution distributors'), 
    ARRAY_CONSTRUCT('supermarket', 'restaurant', 'butcher_shop'), 
    ARRAY_CONSTRUCT('Can deliver Fresh Food', 'Has a Fridge', 'Premium Delivery')
UNION ALL
SELECT
    'Cosmetics', 
    'Hair Products', 
    'Electronic Goods', 
    'Make-up', 
    ARRAY_CONSTRUCT('hair cosmetics make-up beauty'),
    ARRAY_CONSTRUCT('wholesaler warehouse factory supplies distribution distributors'), 
    ARRAY_CONSTRUCT('supermarket', 'outlet', 'fashion'), 
    ARRAY_CONSTRUCT('Can deliver Fresh Food', 'Has a Fridge', 'Premium Delivery')

    ;
SELECT * FROM VEHICLE_ROUTING_SIMULATOR.DATA.LOOKUP

#### Here is an example of how the lookup workes for healthcare for flammable products

In [ ]:
select * from VEHICLE_ROUTING_SIMULATOR.DATA.JOB_TEMPLATE a 

inner join

(select PA from VEHICLE_ROUTING_SIMULATOR.DATA.LOOKUP where industry = 'healthcare') b

on PRODUCT = 'pa'

## Streamlit App

Now you have setup the functions and standing data tables, you can now have a go at running the streamlit app.  The streamlit application is an example of what a basic route optimization simulator could look like.  Feel free to make any changes as you see fit.

- Within Projects / Streamlit you should see a new application
- Click on the app to run the route optimization simulator